In [3]:
import pandahouse as ph
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

sns.set(
    font_scale=1.2,
    style="whitegrid",
    rc={'figure.figsize':(12,7)}
        )

Вы пришли работать стажером-аналитиком и вас просят сделать некоторые выводы о приложении для поиска работы и трафике, который в него закупается.

### Вопросы

* 1. Мы хотим узнать, насколько хорошо наши пользователи-кандидаты проходят первый шаг в приложении – делают отклик на понравившуюся вакансию. Какой процент кандидатов хотя бы раз откликался на вакансию?
* 2. Так как ситуация с безработицей в разных регионах очень разная, мы хотим посмотреть, в каком регионе наибольшее кол-во вакансий. Какой регион имеет больше всего вакансий (регионом вакансии принять регион пользователя)?
* 3. Для того, чтобы предлагать нашим пользователям подходящие вакансии, мы хотим понять, насколько часто люди ищут работу в разных сферах. Какой процент пользователей среди откликавшихся сделал второй отклик на профессию, отличную от профессии первого отклика?
* 4. Сколько пользователей сделали повторный отклик в течение недели после первого?
* 5. Мы закупаем много трафика, но не весь он – релевантный. Какая рекламная сетка нам льет больше всего несовершеннолетних девушек не из РФ?
* 6. Так как у нас в приложении две очень разные группы пользователей – кандидаты и рекрутеры – мы хотим понять, какая рекламная сетка эффективней всего в привлечении сложной аудитории – рекрутеров. Какая рекламная сетка льет нам наибольшую долю рекрутеров?
* 7. Кандидаты тоже различаются по своей активности в приложении – нам интересны те, которые активно откликаются на вакансии, и мы хотим вливать больше денег в те кампании, которые привлекают активных кандидатов. С какой кампании достигается максимальное количество откликов на кандидата?


### Описание данных

Перед вами 3 таблицы с данными:

1.     Users

    * id – идентификатор юзера (UInt32)
    * created_at – дата и время регистрации юзера в базе (DateTime)
    * user_type – 0 если юзер – кандидат, 1 – если рекрутер (UInt8)
    * birthday – год рождения пользователя (Date)
    * sex – 1 если мужчина, 0 если женщина (UInt8)
    * nationality_id – 0 если юзер – РФ, 1 если юзер – не РФ (UInt8)
    * region – регион пользователя (UInt8)
    * adnetwork_n – номер рекламной сетки, из которой пришел пользователь
    * campaign_n – номер рекламной кампании, с которой пришел пользователь

2.     Jobs

    * id – идентификатор вакансии (UInt32)
    * created_at – дата и время создания вакансии в базе (DateTime)
    * user_id – идентификатор пользователя, создавшего вакансию (UInt32)
    * profession – профессия данной вакансии (String)

3.     Applications (отклики на вакансию)

    * id – идентификатор отклика (UInt32)
    * created_at – дата и время создания отклика (DateTime)
    * job_id – идентификатор вакансии, на которую пришел отклик (UInt32)
    * candidate_id – идентификатор кандидата (UInt32)


In [7]:
# Прежде чем приступить к выполнению задани, подключимс к безе данных с таблицами.
connection = dict(database='default',
                  host='https://clickhouse.lab.karpov.courses',
                  user='student',
                  password='dpo_python_2020')

### 1 
Мы хотим узнать, насколько хорошо наши пользователи-кандидаты проходят первый шаг в приложении – делают отклик на понравившуюся вакансию. Какой процент кандидатов хотя бы раз откликался на вакансию?

In [13]:
q_1 = '''
    WITH(SELECT count(*)
    FROM default.users
    WHERE user_type = 0) AS count_candidates
SELECT
    count_candidates,
    uniqExact(candidate_id) AS count_candidates_answered,
    count_candidates_answered/count_candidates*100 AS answered_percent
FROM default.applications
    '''

# отправляем запрос и записываем результат в пандасовский датафрейм
df_1 = ph.read_clickhouse(q_1, connection=connection)
df_1

,count_candidates,count_candidates_answered,answered_percent
0,450,385,85.555556


Всего кандидатов:  450, откликнувшихся кандидатов 385, процент откликнувшихся кандидатов 85.56%.

### 2
Так как ситуация с безработицей в разных регионах очень разная, мы хотим посмотреть, в каком регионе наибольшее кол-во вакансий. Какой регион имеет больше всего вакансий (регионом вакансии принять регион пользователя)?

In [14]:
q_2 = '''
    SELECT
    count(l.user_id) AS count_vacancy,
    r.region
FROM default.jobs AS l
JOIN default.users AS r
    ON l.user_id=r.id
WHERE r.user_type=1
GROUP BY r.region
ORDER BY count_vacancy DESC
    '''

# отправляем запрос и записываем результат в пандасовский датафрейм
df_2 = ph.read_clickhouse(q_2, connection=connection)
df_2

,count_vacancy,region
0,37,0
1,14,7
2,13,2
3,11,5
4,7,1
5,5,6
6,4,8
7,3,3


Наибольшее количество вакансий (37) имеет регион 0.

### 3
Для того, чтобы предлагать нашим пользователям подходящие вакансии, мы хотим понять, насколько часто люди ищут работу в разных сферах. Какой процент пользователей среди откликавшихся сделал второй отклик на профессию, отличную от профессии первого отклика?

In [15]:
q_3 = '''
    WITH(SELECT uniqExact(candidate_id)
    FROM default.applications) AS Candidates
SELECT
    Candidates,
    COUNT(candidate_id) AS Candidates_second,
    (Candidates_second/(Candidates)*100) AS Persentage
FROM(  
    SELECT
        l.candidate_id,
        uniqExact(r.profession) AS unique_profession
    FROM default.applications as l
    JOIN default.jobs as r
    ON l.job_id = r.id
    GROUP BY l.candidate_id
        HAVING unique_profession >= 2)
    '''

# отправляем запрос и записываем результат в пандасовский датафрейм
df_3 = ph.read_clickhouse(q_3, connection=connection)
df_3

,Candidates,Candidates_second,Persentage
0,385,282,73.246753


На 2 и более уникальные профессии 282 из 385 откликнувшихся, что составляет 73.25%

### 4
Сколько пользователей сделали повторный отклик в течение недели после первого?

In [16]:
q_4 = '''
   SELECT
    COUNT(candidate_id) AS Count
FROM
    (SELECT
        candidate_id,
        groupArray(created_at) AS Dates,
        dateDiff('day', Dates[1], Dates[2]) AS diff
    FROM(
        SELECT
            created_at,
            candidate_id
        FROM default.applications
        ORDER BY created_at)
    GROUP BY candidate_id
        HAVING diff BETWEEN 0 AND 7)
    '''

# отправляем запрос и записываем результат в пандасовский датафрейм
df_4 = ph.read_clickhouse(q_4, connection=connection)
df_4

,Count
0,29


29 пользователей сделали повторный отклик в течении недели.

### 5
Мы закупаем много трафика, но не весь он – релевантный. Какая рекламная сетка нам льет больше всего несовершеннолетних девушек не из РФ?

In [17]:
q_5 = '''
  SELECT
    adnetwork_n,
    COUNT(age) AS Count
FROM
    (SELECT
        adnetwork_n,
        sex,
        birthday,
        nationality_id,
        created_at,
        dateDiff('year', birthday, created_at) AS age
    FROM default.users
    WHERE sex = 0 AND age < 18 AND nationality_id = 1)
 GROUP BY adnetwork_n   
 ORDER BY Count DESC
    '''

# отправляем запрос и записываем результат в пандасовский датафрейм
df_5 = ph.read_clickhouse(q_5, connection=connection)
df_5

,adnetwork_n,Count
0,3,5
1,1,4
2,2,3
3,4,1
4,5,1


 Рекламная сетка №3 льет нам больше всего (5) несовершеннолетних девушек не из РФ.

### 6
Так как у нас в приложении две очень разные группы пользователей – кандидаты и рекрутеры – мы хотим понять, какая рекламная сетка эффективней всего в привлечении сложной аудитории – рекрутеров. Какая рекламная сетка льет нам наибольшую долю рекрутеров?

In [19]:
q_6 = '''
  WITH(SELECT
    uniqExact(id)
    FROM default.users
    WHERE user_type=1) AS All_recruiters
SELECT
    adnetwork_n,
    count(adnetwork_n) AS Count_recruiters,
    Count_recruiters/All_recruiters *100 AS Percentage
FROM default.users
GROUP BY adnetwork_n
    HAVING user_type=1
ORDER BY Count_recruiters DESC
    '''

# отправляем запрос и записываем результат в пандасовский датафрейм
df_6 = ph.read_clickhouse(q_6, connection=connection)
df_6

,adnetwork_n,Count_recruiters,Percentage
0,1,15,30.612245
1,3,14,28.571429
2,2,11,22.448980
3,4,8,16.326531
4,5,1,2.040816


 Рекламная сетка # 1 льет нам наибольшую долю рекрутеров - 15, что составляет 30.61% от их общего числа.

### 7
Кандидаты тоже различаются по своей активности в приложении – нам интересны те, которые активно откликаются на вакансии, и мы хотим вливать больше денег в те кампании, которые привлекают активных кандидатов. С какой кампании достигается максимальное количество откликов на кандидата?

In [22]:
q_7 = '''
 SELECT
    campaign_n,
    SUM(count_responses) AS all_responses,
    uniqExact(candidate_id) AS count_candidates,
    all_responses/count_candidates AS responses_per_candidate
FROM
    (SELECT
        default.applications.candidate_id,
        default.users.campaign_n,
        count(default.applications.id) AS count_responses
    FROM default.applications
    JOIN default.users
    ON default.users.id = default.applications.candidate_id
    GROUP BY default.applications.candidate_id, default.users.campaign_n)
GROUP BY campaign_n
ORDER BY responses_per_candidate DESC
    '''

# отправляем запрос и записываем результат в пандасовский датафрейм
df_7 = ph.read_clickhouse(q_7, connection=connection)
df_7.head(1)

,campaign_n,all_responses,count_candidates,responses_per_candidate
0,33,41,11,3.727273


Максимальное количество откликов на кандидата: 3.73 (41 отклик на 11 кандидатов)  достигается от рекламной кампании №33.